In [2]:
import pandas as pd
import krippendorff

In [ ]:
#http://ict.usc.edu/pubs/Inter-Coder%20Agreement%20for%20Computational%20Linguistics.pdf
#https://towardsdatascience.com/inter-rater-agreement-kappas-69cd8b91ff75

### Calculating reliability on final training set (N = 19)

In [212]:
# defining functions

def recode_maintopics(df):
    ''' returns recoded main topics '''

    df['C1: V2'] = df['C1: V2'].str[:-4]
    df['C2: V2'] = df['C2: V2'].str[:-4]
    df['C3: V2'] = df['C3: V2'].str[:-4]
    return df

def get_percent_agreement(x):
    if x[df.columns[0]] == x[df.columns[1]] == x[df.columns[2]]:
         return(100)
    elif x[df.columns[0]] == x[df.columns[1]] != x[df.columns[2]]:
        return(66.66)
    elif x[df.columns[0]] != x[df.columns[1]] == x[df.columns[2]]:
        return(66.66)
    elif x[df.columns[0]] == x[df.columns[2]] != x[df.columns[1]]:
        return(66.66)
    else: 
        return(0)

def get_percent_agreement_frames(df):
    df['1s'] = df.isin([1.0]).sum(axis=1)
    df['1s'] = df['1s'].replace(0, 3)
    df['agreement'] = (df['1s'] / 3 ) * 100
    return df['agreement'].mean()

def recode_maintopics(df):
    ''' returns recoded main topics '''

    df['C1: V2'] = df['C1: V2'].str[:-4]
    df['C2: V2'] = df['C2: V2'].str[:-4]
    df['C3: V2'] = df['C3: V2'].str[:-4]
    return df

#### get Kalpa and % agreement for frames

In [197]:
# Frames

df = pd.read_excel("data/raw/RPA_frames_ICR.xlsx", skiprows=1)
df.columns

#attribution = df[['C1: V4.1', 'C2: V4.1', 'C3: V4.1']][:20].transpose()
print("Attribution of responsibiity:")
attribution = df[['C1: V4.1', 'C2: V4.1', 'C3: V4.1']][:20].transpose()
print("Krippendorff:")
print(krippendorff.alpha(attribution))
print("Percentage agreement:")
print(get_percent_agreement_frames(attribution.transpose()) )

print()

print("Human Interest:")
humaninterest = df[['C1: V5.1', 'C2: V5.1', 'C3: V5.1']][:20].transpose()
print("Krippendorff:")
print(krippendorff.alpha(humaninterest))
print("Percentage agreement:")
print(get_percent_agreement_frames(humaninterest.transpose()) )

print()

conflictframe = df[['C1: V6.1', 'C2: V6.1', 'C3: V6.1']][:20].transpose()
print("Conflict Frame:")
print(krippendorff.alpha(conflictframe))
print("Percentage agreement:")
print(get_percent_agreement_frames(conflictframe.transpose()) )

print()


economic = df[['C1: V8.1', 'C2: V8.1', 'C3: V8.1']][:20].transpose()
print("Economic consequences frame")
print(krippendorff.alpha(economic ))
print("Percentage agreement:")
print(get_percent_agreement_frames(economic.transpose()) )


Attribution of responsibiity:
Krippendorff:
0.3572984749455338
Percentage agreement:
84.99999999999999

Human Interest:
Krippendorff:
0.5809659090909092
Percentage agreement:
86.66666666666666

Conflict Frame:
0.5745192307692308
Percentage agreement:
93.33333333333333

Economic consequences frame
0.7882775119617225
Percentage agreement:
91.66666666666666


#### get Kalpa and % agreement for topics

In [201]:
df = pd.read_excel("data/raw/RPA_topics_ICR.xlsx", skiprows=1)
df = df[['C1: V2', 'C2: V2', 'C3: V2']][:19].astype(str)

df = recode_maintopics(df)
df = df[['C1: V2', 'C2: V2', 'C3: V2']].astype(int)
df = df.transpose()

print("Krippendorff alpha for topics is:")
print(krippendorff.alpha(df))

# Calculate percentage agreement
df = df.transpose()
               
df['percent_agreement'] = df.apply(lambda row: get_percent_agreement(row), axis=1)
print("\nAnd the percentage agreement:")
print(df['percent_agreement'].mean())

Krippendorff alpha for topics is:
0.6227780905654308

And the percentage agreement:
89.47157894736843


#### Getting reliability based on duplicate dataset

In [204]:
df2 = pd.read_csv("data/raw/RPA_coding.csv", skiprows = 1)

In [205]:
df2['n'] = 1
df2.rename(columns={'Document nummer (voor kranten staat in bestand voor kamervragen bijvoorbeeld ah-tk-20012002-89 (z...':'docnr'}, 
                 inplace=True)
df2.columns
d = df2[df2.duplicated('docnr',keep=False)]

In [208]:
d = d[d.Codeursnaam.notnull()]
d = d[d.docnr.notnull()]
d[['Codeursnaam', 'docnr']]

icr = d
icr = icr[~icr.duplicated(['docnr', 'Codeursnaam'])]

icr.sort_values('docnr')[['Codeursnaam', 'docnr']]

icr = icr[~icr.duplicated(['docnr', 'Codeursnaam'])]
icr = icr[icr.duplicated('docnr',keep=False)]

#icr[['Codeursnaam', 'docnr', icr.columns[13]]].sort_values('docnr')

#icr = icr[icr['Krantenbericht of kamervraag?'] == 1]
#icr = icr[icr.duplicated(['docnr', icr.columns[13]],keep=False)]
#icr = icr[icr.duplicated(['docnr', 'Datum? (DD-MM-JJJJ)'],keep=False)]
#icr[icr.duplicated(['docnr', icr.columns[13]],keep=False)][['Codeursnaam', 'docnr', icr.columns[13], 'Datum? (DD-MM-JJJJ)']].sort_values('docnr')

In [209]:
conflict = icr.columns[26]
human_interest = icr.columns[23]
att_res = icr.columns[20]
economic = icr.columns[29]

icr.rename(columns={human_interest : 'human_interest', att_res : 'attribution_resp', conflict : 'conflict',
                    economic : 'economic_consequences', 'Datum? (DD-MM-JJJJ)' : 'publication_date'}, inplace=True)

# get dates

error = icr.publication_date[pd.isnull(pd.to_datetime(icr.publication_date, errors ='coerce'))]
indexes_to_keep = set(range(icr.shape[0])) - set(error.index)
icr = icr.take(list(indexes_to_keep))
icr['publication_date'] = pd.to_datetime(icr.publication_date, errors ='coerce')
icr['year'] = pd.DatetimeIndex(icr['publication_date']).year
icr['month'] = pd.DatetimeIndex(icr['publication_date']).month
icr['day'] = pd.DatetimeIndex(icr['publication_date']).day

icr = icr[icr.duplicated(['docnr','year','month', icr.columns[13]],keep=False)]

#icr = icr[icr.duplicated(['docnr', 'day'],keep=False)]
#icr[icr.duplicated(['docnr', icr.columns[13]],keep=False)][['Codeursnaam', 'docnr', 'year', 'month',icr.columns[13], 'year']].sort_values('docnr')

# make sure doc nr and newspaper are correct
#icr = icr[icr.duplicated(['docnr',  icr.columns[13]],keep=False)]
len(icr)

832

In [210]:
## icr = icr[icr['Krantenbericht of kamervraag?'] == 1]
def get_reliability(df):
    d = {}
    result = []
    for frame in ['human_interest', 'attribution_resp', 'conflict', 'economic_consequences']:
        
        r = icr.pivot(index='docnr', columns='Codeursnaam', values=frame)       
        r['codeurs'] = 5 - r.isnull().sum(axis=1)
        r = r[r['codeurs'] > 1]  
        r = r[[1.0, 3.0, 4.0, 5.0, 6.0]]
        
        print(len(r))
        
        r = r.transpose()
        
        #r_ = icr.pivot(index='docnr', columns='Codeursnaam', values=frame).transpose()
        #print("length of the df: {}".format(len(list(r_.columns)) ))
        kr = krippendorff.alpha(r)

        r = icr.pivot(index='docnr', columns='Codeursnaam', values=frame)
        r['1s'] = r.isin([1.0]).sum(axis=1)
        r['2s'] = r.isin([2.0]).sum(axis=1)
        r['codeurs'] = 5 - r.isnull().sum(axis=1)
        r['agreement'] = ( r['2s'] / r['codeurs'] ) * 100
        d = {"frame": frame,
            "Krippendorf": kr,
            "agreement": r['agreement'].mean() }
        result.append(d)
    return result

result = get_reliability(icr)
result

395
15
395
395


[{'frame': 'human_interest',
  'Krippendorf': 0.07485975212002605,
  'agreement': 81.66666666666666},
 {'frame': 'attribution_resp',
  'Krippendorf': 0.011363636363636354,
  'agreement': 97.33009708737865},
 {'frame': 'conflict',
  'Krippendorf': 0.14824905203454097,
  'agreement': 81.0337552742616},
 {'frame': 'economic_consequences',
  'Krippendorf': 0.3644119833241768,
  'agreement': 82.51054852320675}]

In [211]:
def label_topic(row):
    if row['main_topic'] == '1':
        return 'Macro-economie en belastingen'
    if row['main_topic'] == '2':
        return 'Burgerrechten en vrijheden'
    if row['main_topic'] == '3':
        return 'Gezondheid'
    if row['main_topic'] == '4':
        return 'Landbouw en Visserij'
    if row['main_topic'] == '5':
        return 'Arbeid'
    if row['main_topic'] == '6':
        return 'Onderwijs'
    if row['main_topic'] == '7':
        return 'Milieu'
    if row['main_topic'] == '8':
        return 'Energiebeleid'
    if row['main_topic'] == '9':
        return 'Immigratie en integratie'
    if row['main_topic'] == '10':
        return 'Verkeer en vervoer'
    if row['main_topic'] == '11':
        return 'Unkown'
    if row['main_topic'] == '12':
        return 'Justitie, Rechtspraak, Criminaliteit'
    if row['main_topic'] == '13':
        return 'sociale Zaken'
    if row['main_topic'] == '14':
        return 'Gemeenschapsontwikkeling, huisvestingsbeleid en stedelijke planning'
    if row['main_topic'] == '15':
        return 'Ondernemingen, Bankwezen en binnenlandse handel '
    if row['main_topic'] == '16':
        return 'Defensie'
    if row['main_topic'] == '17':
        return 'Wetenschappelijk onderzoek, technologie en communicatie'
    if row['main_topic'] == '18':
        return 'Buitenlandse handel'
    if row['main_topic'] == '19':
        return 'Buitenlandse zaken en ontwikkelingssamenwerking'
    if row['main_topic'] == '20':
        return 'Functioneren democratie en openbaar bestuur'
    if row['main_topic'] == '21':
        return 'Ruimtelijke ordening, publiek natuur- en waterbeheer'
    if row['main_topic'] == '22':
        return 'Unkown 2'
    if row['main_topic'] == '23':
        return 'Kunst, cultuur en entertainment'
    if row['main_topic'] == '00':
        return 'Toegevoegde codes voor media'


def recode_maintopics(df):
    ''' returns recoded main topics '''

    df['main_topic'] = df['topic_number'].str[:-2]
    df['main_topic_label'] = df.apply (lambda row: label_topic (row),axis=1)
    df[['main_topic','main_topic_label']][0:5]
    return df


icr['topic_number'] = icr[icr.columns[17]]
df = recode_maintopics(icr)

df['main_topic'] = pd.to_numeric(df['main_topic'], errors='coerce')
r = df.pivot(index='docnr', columns='Codeursnaam', values='main_topic')
#print("length of the df: {}".format(len(list(r_.columns)) ))
r = r.values

krippendorff.alpha(r)
#import numpy as np
#df['main_topic'] = df['main_topic'].replace('', np.nan)
#df['main_topic'].astype(int)

0.0024608866213596814